#Initialization of the kaggle and the files#

In [5]:
# reading in the data via the Kaggle API

# mount your Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
# install Kaggle
! pip install kaggle

Log into your Kaggle account, go to Account > API > Create new API token. You will obtain a kaggle.json file, which you save on your Google Drive directy in my drive.

In [7]:
!mkdir ~/.kaggle

In [8]:
#read in your Kaggle credentials from Google Drive
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/kaggle.json


In [9]:
! kaggle competitions download -c detecting-french-texts-difficulty-level-2023
from zipfile import ZipFile
with ZipFile('detecting-french-texts-difficulty-level-2023.zip','r') as zip:
  zip.extractall(path="")

  0% 0.00/303k [00:00<?, ?B/s]
100% 303k/303k [00:00<00:00, 112MB/s]


# Begining of the model#

In [10]:
!pip install wikipedia
!pip install -q transformers
!python -m spacy download fr_core_news_sm


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=e28dc16657259acb08457af71b5254c6c9942a020fb1da3a28abca1f9c8c4ea5
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.3 MB/s eta 0:00:00
2023-11-15 16:20:14.801679: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 16:20:14.801753: E tensorflow/compiler/xl

## Imports##

In [11]:
# read in your training data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
df = pd.read_csv('training_data.csv', index_col = 'id')

In [12]:
# Import standard libraries
import pandas as pd
import numpy as np
import math
import bs4 as bs
import urllib.request
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Import for text analytics
import spacy
from spacy import displacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import wikipedia
import gensim
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.utils import simple_preprocess
from gensim import corpora
import multiprocessing

# Import libraries for logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

# Import libraries for hugginface
from transformers import pipeline
import gensim.downloader

In [13]:
sp =spacy.load('fr_core_news_sm')

In [14]:
df

,sentence,difficulty
id,,
0,Les coûts kilométriques réels peuvent diverger...,C1
1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,Le test de niveau en français est sur le site ...,A1
3,Est-ce que ton mari est aussi de Boston?,A1
4,"Dans les écoles de commerce, dans les couloirs...",B1
...,...,...
4795,"C'est pourquoi, il décida de remplacer les hab...",B2
4796,Il avait une de ces pâleurs splendides qui don...,C1
4797,"Et le premier samedi de chaque mois, venez ren...",A2


In [15]:
X=df['sentence']
y=df['difficulty']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])


In [18]:
parameters = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'tfidf__min_df': [1, 5],
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'clf__C': [0.1, 1, 10, 100],
    'clf__max_iter': [100, 1000, 2000]
}


In [19]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, cv=5)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'clf__C': [0.1, 1, 10, 100],
                         'clf__max_iter': [100, 1000, 2000],
                         'tfidf__max_df': [0.5, 0.75, 1.0],
                         'tfidf__min_df': [1, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]})

In [27]:
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_params

{'clf__C': 100,
 'clf__max_iter': 1000,
 'tfidf__max_df': 0.5,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 2)}

In [44]:
def spacy_tokenizer(sentence):

    punctuations = string.punctuation
    #stop_words=['le','la','les','un','une','des','à','et','dans','dehors','droite','gauche']
    #stop_words = spacy.lang.fr.stop_words.STOP_WORDS
    #list_no_stop_words=['cependant','néanmoins','autrement','auxquels','ouste','anterieur','desquelles','notamment','puisque','hormis','lesquels','devers','relativement','quant-à-soi','revoici','envers','malgré','procedant','elles-mêmes','spécifiques','vôtres','revoila','celles-ci','quiconque','delà']
    #stop_words2=[x for x in stop_words if x not in list_no_stop_words]

    # Create token object, which is used to create documents with linguistic annotations.
    mytokens = sp(sentence)

    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Remove stop words and punctuation
    #mytokens = [ word for word in mytokens if word not in stop_words ]

    # Remove anonymous dates and people
    mytokens = [ word.replace('xx/', '').replace('xxxx/', '').replace('xx', '') for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in ["xxxx", "xx", ""] ]

    # Return preprocessed list of tokens
    return mytokens

In [21]:
#stop_words = spacy.lang.fr.stop_words.STOP_WORDS
#print('Number of stopwords: %d' % len(stop_words))
#print(stop_words)
#list_no_stop_words=['cependant','néanmoins','autrement','auxquels','ouste','anterieur','desquelles','notamment','puisque','hormis','lesquels','devers','relativement','quant-à-soi','revoici','envers','malgré','procedant','elles-mêmes','spécifiques','vôtres','revoila','celles-ci','quiconque','delà']
#stop_words2=[x for x in stop_words if x not in list_no_stop_words]
#print('Number of stopwords: %d' % len(stop_words2))

In [45]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1,max_df=0.5, norm='l2', encoding='UTF-8', ngram_range=(1, 2), tokenizer=spacy_tokenizer)

# Define classifier
classifier = LogisticRegression(C=1.0,solver='lbfgs', max_iter=1000,)

# Create pipeline
pipe = Pipeline([('vectorizer', tfidf),
                 ('classifier', classifier)])

# Fit model on training set
pipe.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(encoding='UTF-8', max_df=0.5,
                                 ngram_range=(1, 2), sublinear_tf=True,
                                 tokenizer=<function spacy_tokenizer at 0x7b0792c22b90>)),
                ('classifier', LogisticRegression(max_iter=1000))])

In [46]:
y_pred = pipe.predict(X_test)

In [47]:
y_pred

array(['A1', 'A1', 'C2', 'B1', 'A1', 'C2', 'A1', 'A2', 'A2', 'B2', 'A1',
       'A2', 'C2', 'B2', 'C2', 'C2', 'B1', 'C2', 'A1', 'A1', 'A1', 'C1',
       'A2', 'C1', 'A1', 'A2', 'B1', 'A1', 'C2', 'A1', 'B2', 'B2', 'A2',
       'B1', 'A2', 'B2', 'A2', 'C2', 'C1', 'A2', 'C2', 'C2', 'C2', 'A2',
       'B1', 'B1', 'A1', 'C2', 'B2', 'C1', 'B2', 'A2', 'C2', 'A2', 'C1',
       'A2', 'C2', 'C2', 'A2', 'B1', 'C1', 'C2', 'A2', 'C2', 'C1', 'C1',
       'C2', 'A2', 'C2', 'B1', 'C2', 'B2', 'C2', 'B2', 'A2', 'C2', 'A1',
       'A1', 'A1', 'A1', 'A2', 'B2', 'C1', 'C1', 'C1', 'A1', 'A1', 'C2',
       'C2', 'A1', 'C2', 'C2', 'B2', 'A1', 'A2', 'B1', 'A2', 'A1', 'B1',
       'C1', 'C2', 'C2', 'C2', 'A1', 'B1', 'A1', 'A1', 'C1', 'B1', 'B1',
       'C1', 'A2', 'A2', 'A1', 'C1', 'A1', 'C1', 'C1', 'B1', 'A1', 'A1',
       'A1', 'B1', 'C2', 'C2', 'A2', 'C1', 'B2', 'B1', 'B1', 'C1', 'C2',
       'A1', 'B1', 'A1', 'C2', 'C1', 'A2', 'A1', 'C2', 'A2', 'B2', 'B1',
       'B1', 'A1', 'A2', 'C2', 'A1', 'A2', 'C2', 'B

In [48]:
accuracy_tfidf = round(accuracy_score(y_test, y_pred), 4)
print(f'The accuracy using TF-IDF is: {accuracy_tfidf}')

The accuracy using TF-IDF is: 0.4677


In [49]:
df_pred = pd.read_csv('unlabelled_test_data.csv', index_col = 'id')
df_pred

,sentence
id,
0,Nous dûmes nous excuser des propos que nous eû...
1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,"Et, paradoxalement, boire froid n'est pas la b..."
3,"Ce n'est pas étonnant, car c'est une saison my..."
4,"Le corps de Golo lui-même, d'une essence aussi..."
...,...
1195,C'est un phénomène qui trouve une accélération...
1196,Je vais parler au serveur et voir si on peut d...
1197,Il n'était pas comme tant de gens qui par pare...


In [50]:
df_pred['predicted_difficulty'] = pipe.predict(df_pred['sentence'])

In [51]:
df_pred

,sentence,predicted_difficulty
id,,
0,Nous dûmes nous excuser des propos que nous eû...,A2
1,Vous ne pouvez pas savoir le plaisir que j'ai ...,A2
2,"Et, paradoxalement, boire froid n'est pas la b...",B1
3,"Ce n'est pas étonnant, car c'est une saison my...",B1
4,"Le corps de Golo lui-même, d'une essence aussi...",C2
...,...,...
1195,C'est un phénomène qui trouve une accélération...,B1
1196,Je vais parler au serveur et voir si on peut d...,A2
1197,Il n'était pas comme tant de gens qui par pare...,C2


# This is how to submit a pd file with predictions
### for the example we will submit a file where only A1 is given as a prediction

In [37]:
df_sub=df_pred[['predicted_difficulty']]
df_sub.to_csv('submission.csv')

In [38]:
! kaggle competitions submit -c detecting-french-texts-difficulty-level-2023 -f submission.csv -m "Sample submission"

100% 8.31k/8.31k [00:01<00:00, 8.37kB/s]
Successfully submitted to Detecting the difficulty level of French texts